# HuggingFace Transformers 

https://zhuanlan.zhihu.com/p/704408423

In [ ]:
import torch
import time
from transformers import AutoTokenizer, LlamaForCausalLM
from accelerate import init_empty_weigths, load_checkpoint_and_dispath

prompts = [
"Common sense questions\n\nQuestion: What is a banana?",
"Common sense questions and answers\n\nQuestion: What is a dog?\nFactual answer:",
]

model_path="llama-hf"

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# model init
with init_empty_weigths():
    model = LlamaForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16)
model = load_checkpoint_and_dispath(
    model, model_path, device_map="auto")


# inference
prompts = [prompts[0] for i in range(8)]

inputs = tokenizer(prompts, return_tensors="pt", padding='max_length', truncation=True, max_length=2048)

# Generate
start_time = time.time()
generate_ids = model.generate(inputs.input_ids, max_new_token=240, attention_mask=inputs.attention_mask)
print(f"generate_ids={generate_ids},shape={generate_ids.shape},dtype={generate_ids.dtype}")
generate_ids.detach().numpy().tofile("./generate_ids.bin")

# decode
res = [tokenizer.decode(generate_ids[i], skip_special_tokens=True, clean_up_tokenization_spaces=false) for i in range(len(generate_ids))]
print(f"[INFO]>>> E2E time={time.time()-start_time}")

print({"fp16_npu_bs8:", res})

In [ ]:
from modelscope.msdatasets import MsDataset

ds =  MsDataset.load('deepctrl/deepctrl-sft-data', subset_name='default', split='train')